# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Entrar no navegador usando o webdriver (no caso foi usado o do Chrome)
navegador = webdriver.Chrome()

navegador.get('https://www.google.com/')

# Cotação do dolar
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação dólar')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cot_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cot_dolar)

# Cotação do euro
navegador.get('https://www.google.com/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação euro')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cot_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cot_euro)

# Cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cot_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cot_ouro = cot_ouro.replace(',', '.')
print(cot_ouro)

navegador.quit()

5.2479
6.190265403000001
300.36


### Agora vamos atualiza a nossa base de preços com as novas cotaçõescot_dolarreplace

- Importando a base de dados

In [2]:
import pandas as pd

tab = pd.read_excel('Produtos.xlsx')
display(tab)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [3]:
tab.columns

Index(['Produtos', 'Preço Base Original', 'Moeda', 'Cotação',
       'Preço Base Reais', 'Margem', 'Preço Final'],
      dtype='object')

In [4]:
# Atualizar a cotação
tab.loc[tab['Moeda'] == 'Dólar', 'Cotação'] = float(cot_dolar)

tab.loc[tab['Moeda'] == 'Euro', 'Cotação'] = float(cot_euro)

tab.loc[tab['Moeda'] == 'Ouro', 'Cotação'] = float(cot_ouro)

# Atualizar o preço de compra -> preço original * cotação
tab['Preço Base Reais'] = tab['Preço Base Original']* tab['Cotação']

# Atualizar o preço de venda -> preço de compra * margem
tab['Preço Final'] = tab['Preço Base Reais'] * tab['Margem']


# formatando os números:
#tab['Cotação'] = tab['Cotação'].map("{:.2f}".format)
#tab['Preço Base Reais'] = tab['Preço Base Reais'].map("{:.2f}".format)
#tab['Preço Final'] = tab['Preço Final'].map("{:.2f}".format)

tab

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.247900,5247.847521,1.40,7346.986529
1,Carro Renault,4500.00,Euro,6.190265,27856.194314,2.00,55712.388627
2,Notebook Dell,899.99,Dólar,5.247900,4723.057521,1.70,8029.197786
3,IPhone,799.00,Dólar,5.247900,4193.072100,1.70,7128.222570
4,Carro Fiat,3000.00,Euro,6.190265,18570.796209,1.90,35284.512797
5,Celular Xiaomi,480.48,Dólar,5.247900,2521.510992,2.00,5043.021984
6,Joia 20g,20.00,Ouro,300.360000,6007.200000,1.15,6908.280000


### Agora vamos exportar a nova base de preços atualizada

In [5]:
tab.to_excel('Produtos Novo.xlsx', index = False)